In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp
from rsome import ro
from rsome import grb_solver as grb
import rsome as rso
import numpy as np

import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import pandas as pd

In [2]:
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.family'] = 'STIXGeneral'

SMALL_SIZE = 12
MEDIUM_SIZE = 14
BIGGER_SIZE = 18

plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=BIGGER_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [3]:
n = 20 # number of items (c in R^n)
d = 10 # dim of context to utility (x in R^d)

In [4]:
theta = np.random.randint(low=0, high=2, size=(d, n)) # theta has to be globally fixed to ensure exchangeability

def g(x):
    c = (x @ theta) ** 2
    return c

In [12]:
N = 2_000
N_train = 1000
N_test  = 1000

x_dataset = np.random.uniform(low=0, high=4, size=(N, d))
c_dataset = g(x_dataset) * np.random.uniform(low=4/5, high=6/5, size=(N, n))

device = ("cuda" if torch.cuda.is_available() else "cpu")
to_tensor = lambda r : torch.tensor(r).to(torch.float32).to(device)
x_train, x_cal = to_tensor(x_dataset[:N_train]), to_tensor(x_dataset[N_train:])
c_train, c_cal = to_tensor(c_dataset[:N_train]), to_tensor(c_dataset[N_train:])

x_test = np.random.uniform(low=0, high=4, size=(N_test, d))
c_test = g(x_test) * np.random.uniform(low=4/5, high=6/5, size=(N_test, n))

In [13]:
train_dataset = TensorDataset(x_train, c_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

class FeedforwardNN(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=64):
        super(FeedforwardNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [15]:
model = FeedforwardNN(input_dim=d, output_dim=n).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 1_000

for epoch in range(num_epochs):
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(batch_X)  # Forward pass
        loss = criterion(outputs, batch_y)  # Compute the loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights

    if epoch % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/1000], Loss: 26955.0449
Epoch [101/1000], Loss: 2078.3601
Epoch [201/1000], Loss: 1308.6206
Epoch [301/1000], Loss: 737.9594
Epoch [401/1000], Loss: 419.3353
Epoch [501/1000], Loss: 460.1890
Epoch [601/1000], Loss: 355.9471
Epoch [701/1000], Loss: 280.1278
Epoch [801/1000], Loss: 324.3944
Epoch [901/1000], Loss: 294.9506


In [43]:
def box_solve_generic(c_box_lb, c_box_ub, c_true, p, B):
    covered = int(np.all(c_box_lb <= c_true) and np.all(c_true <= c_box_ub))

    # perform RO over constraint region
    model = ro.Model()

    w = model.dvar(n)
    c = model.rvar(n)
    uset = (c_box_lb <= c, c <= c_box_ub)

    model.minmax(-c @ w, uset)
    model.st(w <= 1)
    model.st(w >= 0)
    model.st(p @ w <= B)

    model.solve(grb)
    return covered, model.get()

In [44]:
# *marginal* box constraint (i.e. just ignore contextual information)
def box_solve_marg(alpha, x, c_true, p, B):
    alpha = alpha / 20
    c_box_lb = np.quantile(c_dataset, q=(alpha / 2), axis=0)
    c_box_ub = np.quantile(c_dataset, q=(1 - alpha / 2), axis=0)
    return box_solve_generic(c_box_lb, c_box_ub, c_true, p, B)

In [45]:
# *conditional* box conformal constraint (PTC-B)
def box_solve_cp(alpha, x, c_true, p, B):
    c_pred = model(x_cal)
    box_cal_scores = np.linalg.norm((c_pred - c_cal).detach().cpu().numpy(), np.inf, axis=1)
    conformal_quantile = np.quantile(box_cal_scores, q=1 - alpha, axis=0)

    c_box_hat = model(to_tensor(x)).detach().cpu().numpy()
    c_box_lb = c_box_hat - conformal_quantile
    c_box_ub = c_box_hat + conformal_quantile
    return box_solve_generic(c_box_lb, c_box_ub, c_true, p, B)

In [46]:
from scipy.stats._continuous_distns import chi2

mu = np.mean(c_dataset, axis=0)
cov = np.cov(c_dataset.T)
sqrt_cov_inv = np.linalg.cholesky(np.linalg.inv(cov))
mah_dists = np.linalg.norm((c_dataset - mu) @ sqrt_cov_inv, axis=1)

# *marginal* ellipsoid constraint
def ellipsoid_solve_marg(alpha, x, c_true, p, B):
    # perform RO over constraint region
    model = ro.Model()

    w = model.dvar(n)
    c = model.rvar(n)

    cutoff = np.quantile(mah_dists, q=1 - alpha)
    uset = rso.norm((c - mu).T @ sqrt_cov_inv, 2) <= cutoff
    covered = int(np.linalg.norm((c_true - mu).T @ sqrt_cov_inv) <= cutoff)

    model.minmax(-c @ w, uset)
    model.st(w <= 1)
    model.st(w >= 0)
    model.st(p @ w <= B)

    model.solve(grb)
    return covered, model.get()

In [47]:
def trial(alpha, idx):
    x = x_test[idx]
    c_true = c_test[idx]

    p = np.random.randint(low=0, high=1000, size=n)
    u = np.random.uniform(low=0, high=1)
    B = np.random.uniform(np.max(p), np.sum(p) - u * np.max(p))

    marg_box_covered, marg_box_value = box_solve_marg(alpha, x, c_true, p, B)
    cp_box_covered, cp_box_value = box_solve_cp(alpha, x, c_true, p, B)
    marg_ellipsoid_covered, marg_ellipsoid_value = ellipsoid_solve_marg(alpha, x, c_true, p, B)
    return (marg_box_covered, marg_box_value), (cp_box_covered, cp_box_value), (marg_ellipsoid_covered, marg_ellipsoid_value)

In [48]:
alphas = [0.05]
coverages = {
    r"$\alpha$": alphas,
    "Box": [],
    "PTC-B": [],
    "Ellipsoid": [],
}
values = {
    r"$\alpha$": alphas,
    "Box": [],
    "PTC-B": [],
    "Ellipsoid": [],
}

n_trials = 100

for alpha in alphas:
    marg_box_covered = 0
    marg_box_values = []

    cp_box_covered = 0
    cp_box_values = []

    marg_ellipsoid_covered = 0
    marg_ellipsoid_values = []

    for trial_idx in range(n_trials):
        (marg_box_covered_trial, marg_box_value_trial), (cp_box_covered_trial, cp_box_value_trial), (marg_ellipsoid_covered_trial, marg_ellipsoid_value_trial) = trial(alpha, trial_idx)
        marg_box_covered += marg_box_covered_trial
        marg_box_values.append(marg_box_value_trial)

        cp_box_covered += cp_box_covered_trial
        cp_box_values.append(cp_box_value_trial)

        marg_ellipsoid_covered += marg_ellipsoid_covered_trial
        marg_ellipsoid_values.append(marg_ellipsoid_value_trial)

    coverages["Box"].append(marg_box_covered / n_trials)
    coverages["PTC-B"].append(cp_box_covered / n_trials)
    coverages["Ellipsoid"].append(marg_ellipsoid_covered / n_trials)

    values["Box"].append(np.mean(marg_box_values))
    values["PTC-B"].append(np.mean(cp_box_values))
    values["Ellipsoid"].append(np.mean(marg_ellipsoid_values))

TypeError: trial() missing 1 required positional argument: 'idx'

In [49]:
import pandas as pd

coverage_df = pd.DataFrame(coverages)
values_df = pd.DataFrame(values)

ValueError: All arrays must be of the same length

In [42]:
print(coverage_df)
print(values_df)

   $\alpha$  Box  PTC-B  Ellipsoid
0      0.05  0.0    0.0       0.94
   $\alpha$  Box  PTC-B     Ellipsoid
0      0.05  NaN    NaN  6.920573e-08


In [54]:
print(coverage_df.to_latex(index=False,
                  formatters={"name": str.upper},
                  float_format="{:.2f}".format,
                  column_format="c" * len(coverage_data),
))

print(values_df.to_latex(index=False,
                  formatters={"name": str.upper},
                  float_format="{:.2f}".format,
                  column_format="c" * len(values_data)
))

NameError: name 'coverage_data' is not defined